<img src="https://raw.githubusercontent.com/determined-ai/determined/master/determined-logo.png" align='right' width=150 />

# Building a Pedestrian Detection Model with Determined

<img src="https://www.cis.upenn.edu/~jshi/ped_html/images/PennPed00071_1.png" width=400 />


This notebook will walk through the benefits of building a Deep Learning model with Determined.  We will build an object detection model trained on the [Penn-Fudan Database for Pedestrian Detection and Segmentation](https://www.cis.upenn.edu/~jshi/ped_html/) and deploy it to Algorithmia for serving.

In [ ]:
!pip install torch==1.4.0 torchvision==0.5.0

In [14]:
import Algorithmia
import torch
import torchvision

from determined.experimental import Determined

# Train

## Train a model on the Determined cluster

For our first example, we run a simple single-GPU training job with fixed hyperparameters.

In [1]:
# DET_MASTER_IP = < Determined Master IP >
DET_MASTER_IP = "http://latest-master.determined.ai:8080"

In [ ]:
!det -m {DET_MASTER_IP} e create const.yaml .

## Determined Model Registry

After training, we'll want to actually use our model in some sort of system.  Determined provides a model registry to version your trained models, making them easy to retrieve for inference.

In [6]:
# experiment_id = < Experiment ID >
experiment_id = 715
MODEL_NAME = "ped-detection"

In [10]:
%%capture
checkpoint = Determined(master=DET_MASTER_IP).get_experiment(experiment_id).top_checkpoint()
model = Determined(master=DET_MASTER_IP).create_model(MODEL_NAME)
model.register_version(checkpoint.uuid)

# Inference

## Local Inference

Once your model is versioned in the model registry, using that model for inference is straightforward:

In [17]:
model = Determined(master=DET_MASTER_IP).get_model(MODEL_NAME)
trial = model.get_version().load()
inference_model = trial.model

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [ ]:
from predict import predict
predict(inference_model, 'test.jpg')

## Serving Endpoint

Now that we can run inference in this notebook, let's set up a serving endpoint on Algorithmia so we can scale this model's serving.

### Save your model locally

First, we'll save the model from the Determined Model Registry.

In [ ]:
torch.save(inference_model, MODEL_NAME)

### Upload your model and sample data

In [ ]:
# API_KEY = <Your Algorithmia API Key>
API_KEY = "simWGAU/QaL4GTvEkSOp6ivjLHC1"
# USERNAME = <Your Algorithmia username>
USERNAME = "hoanghphan"

client = Algorithmia.client(API_KEY)

In [ ]:
DATA_COLLECTION = "pedestrian-detection"

'''
In Algorithmia, the full data directory is pre-pended with 'data://'
but this depends on where your data is stored, e.g. it would be
's3://' if the data is in s3
'''

DIRECTORY = client.dir(f"data://{USERNAME}/{DATA_COLLECTION}")
if DIRECTORY.exists() is False:
    DIRECTORY.create()

Once you've created the data collection, you can upload your model and test image to it:

In [ ]:
# Model
MODEL_PATH = f"data://{USERNAME}/{DATA_COLLECTION}/{MODEL_NAME}"
client.file(MODEL_PATH).putFile(MODEL_NAME)

# Test image
TEST_IMG_PATH = f"data://{USERNAME}/{COLLECTION}/test.jpg"
client.file(TEST_IMG_PATH).putFile("test.jpg")

### Create your serving Algorithm

Algorithmia refers to each endpoint generically as an Algorithm. An Algorithm can be any executable code, in this case it is code that uses our model to generate predictions.

In [ ]:
ALGORITHM_NAME = "Pedestrian-algorithm"

algo = client.algo(f"{USERNAME}/{ALGORITHM_NAME}")
algo.create(
    details = {
        "label": "Object Detection Serving Endpoint",
    },
    settings = {
        "language": "python3-1",
        "source_visibility": "closed",
        "license": "apl",
        "network_access": "full",
        "pipeline_enabled": True,
        "environment": "cpu"
    }
)

### Add Inference Code to your serving Algorithm

Now that the Algorithm is created, we can add the inference code that we want it to execute.